# MLiP - Regression - Feature Expansions
Kurs Maschinelles Lernen in der Produktion  

#### In diesem Notebook wird die Feature Expansion am Beispiel des Verfahren Regression betrachtet.

### 0. Bibliotheken importieren

In [ ]:
# Importiere benötigte Bibliotheken
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline
style = 'seaborn-whitegrid'
plt.style.use(style)
plt.rcParams.update({'font.size': 14})  # Schriftgröße aller Textzeichen im Graphen

### 1. Daten erfassen

In [ ]:
# Erzeuge Datensatz
df = pd.DataFrame({'X': np.array([-2, -1, 0, 1, 2]),
                   'y': np.array([3, 2, 3, 6, 11])
                  })
print('Daten erfolgreich eingelesen')


### 2. Daten erkunden

In [ ]:
# Daten erkunden                 
fig = plt.figure(figsize=(4, 4))
plt.plot(df['X'], df['y'], 'o')
plt.xlabel('X')
plt.ylabel('y')
plt.title('Datensatz')
plt.show()

print('Datensatz')
df

### 3. Daten vorbereiten

In [ ]:
# Daten vorbereiten
X_train = df[['X']]
y_train = df[['y']]
#Erzeuge Gitter für Ausgabe
X_grid = np.linspace(-2.0, 2.0, 100).reshape(100,1)
print('Schritt erfolgreich')

### 4. Modelle bilden

In [ ]:
# Regressionsgerade und Vorhersage
# Importieren der Funktion: LineareRegression aus dem Modul linear_model
from sklearn.linear_model import LinearRegression

# Modell erstellen
regr = LinearRegression()
# Trainieren des Lineares Regressionsmodells
regr = regr.fit(X_train, y_train)
print('Training erfolgreich')

### 5. Modelle validieren

In [ ]:
# Ausgabe Graphen
y_lin_pred = regr.predict(X_grid)  

plt.plot(df['X'], df['y'], 'o')
plt.xlabel('X')
plt.ylabel('y')
plt.title('Datensatz mit Modelgeraden')
plt.plot(X_grid, y_lin_pred)
plt.show()

### Basisraum erweitern

In [ ]:
df2 = df
df2[['phi_2']] = df[['X']] * df[['X']]
df2 = df.rename(index=str, columns={"X": "phi_1"})
df2

In [ ]:
%matplotlib notebook

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df2['phi_1'], df2['phi_2'], df2['y'], 'o', c='k')
ax.text2D(0.05, 0.95, "Datensatz erweitert", transform=ax.transAxes)
ax.set_xlabel('phi_1')
ax.set_ylabel('phi_2')
ax.set_zlabel('y')
plt.show()

In [ ]:
# Daten Vorbereiten
X2_train = df2.drop(columns=['y'])
y_train = df2[['y']]

# Trainieren des Lineares Regressionsmodells
regr_extended = regr.fit(X2_train, y_train)


#Erzeuge Gitter für Ausgabe
phi1 = np.linspace(-2.0, 2.0, 100).reshape(100,1)
phi2 = np.linspace(0.0, 4.0, 100).reshape(100,1)
phi1v, phi2v = np.meshgrid(phi1, phi2)

# Ausgabe Graphen
y_extended_lin_pred = regr_extended.predict(np.array([phi1v.ravel(), phi2v.ravel()]).T).reshape(phi1v.shape)  

%matplotlib notebook

# Kombination aus 3d Scatterplot und 3d Surfaceplot
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df2[['phi_1']], df2[['phi_2']], df2[['y']], 'o', c='k')
ax.plot_surface(phi1v, phi2v, y_extended_lin_pred, cmap=cm.jet)
ax.text2D(0.05, 0.95, "lineare Regression", transform=ax.transAxes)
ax.set_xlabel('phi_1')
ax.set_ylabel('phi_2')
ax.set_zlabel('y')
plt.show()

### 2: quadratische Regression - automatisch

### Modelle bilden & trainieren

In [ ]:
# Regressionsgerade und Vorhersage
# Importieren der Funktion: PolynomialFeatures
# In Kombination anwenden mit LinearRegression von scikit learn

# Daten nochmal holen
X_train = df[['X']]

from sklearn.preprocessing import PolynomialFeatures


# polynomiale Basisfunktion Grad = 2, Objekt bauen und anwenden
quadr_features = PolynomialFeatures(degree = 2)
X_quadr_train = quadr_features.fit_transform(X_train)

# Modell erzeugen und lineare Regression mit X_quadr_train trainieren
regr_extended_linear = LinearRegression()
regr_extended_linear = regr_extended_linear.fit(X_quadr_train, y_train)

# Überprüfung
# Berechne MAE
from sklearn.metrics import mean_absolute_error as mae
regr_mae = mae(y_train, regr_extended_linear.predict(X_quadr_train))
print('Fehlerabweichung (Erweiterung des Basisraums): \t' + str(regr_mae))

#Version 2 eleganter, kürzer und effizienter
from sklearn.pipeline import make_pipeline

# Trainieren des quadratischen Regressionsmodells
regr_quadratic = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
regr_quadratic = regr_quadratic.fit(X_train, y_train)

# Überprüfung
# Berechne MAE
quadr_mae = mae(y_train, regr_quadratic.predict(X_train))
print('Fehlerabweichung (Pipeline): \t\t\t' + str(quadr_mae))


In [ ]:
# Ausgabe Graphen
y_quad_pred = regr_quadratic.predict(X_grid)

%matplotlib inline 
plt.figure(figsize=(9,6))
plt.plot(df['X'], df['y'], 'o', label='Daten')
plt.plot(X_grid, y_quad_pred, label='Regressionsgerade')
plt.xlabel('X')
plt.ylabel('y')
plt.title('Datensatz mit Modelgeraden')
plt.legend()
plt.show()